In [ ]:
import gradio as gr
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool

from docx import Document
from io import BytesIO
import base64

import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] =os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

In [ ]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0513')

In [ ]:
df=pd.read_csv('./diag_5mcids.csv')
df

In [ ]:
df_json = df.to_json()
import json


In [ ]:
df[df['mcid']==1].shape

In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [ ]:
@tool
def patientRecordSelect_tool(df_list, mcid):
    """Select the patient's medical records for patient with given mcid.
    'dos': date of service.
    'cd_value': ICD10 diagnosis codes.
    'ICD_FULL_DESC': description of ICD10 code.
    """
    df = pd.DataFrame(df_list)
    output = df[df['mcid'] == mcid][['dos', 'cd_value', 'ICD_FULL_DESC']]
    return output

patientRecordSelect_agent = Agent(
    role="Get patient's medical records",
    goal="For a given patient identified by mcid, get this patient's medical recordes "
         "from a given data using tool patientRecordSelect_tool.",
    backstory="You use given mcid to find the corresponding medical records.",
    tools=[patientRecordSelect_tool],
    allow_delegation=False,
    llm=llm  #Make sure this variable is defined
)

patientRecordSelect_task = Task(
    description="Get the medical records for the given patient {mcid} from file {df_list}. These medical records will be used in later steps",
    expected_output="a table with columns: date of service, ICD10 diagnosis codes, code description",
    agent=patientRecordSelect_agent
)

summary_agent = Agent(
    role="Medical Record Analyst, Prioritizer, and Summarizer",
    goal="Analyze, prioritize, and summarize patient's medical records, emphasizing the most serious conditions while providing detailed evidence for past, present, and potential future medical issues.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a highly skilled medical analyst with years of experience in critical care and risk assessment, "
        "you excel at thoroughly examining patient records to identify, prioritize, and explain medical conditions. "
        "Your expertise lies in quickly recognizing the most serious health issues and their potential impacts "
        "on a patient's well-being. You're adept at connecting specific medical codes and treatments to diagnosed "
        "conditions, providing a comprehensive understanding of a patient's health journey. Your background in "
        "triage and emergency medicine enhances your ability to rank conditions by severity and urgency. "
        "Additionally, you're skilled at spotting patterns that might indicate future health risks based on "
        "current and past medical data, always keeping an eye on the most critical factors affecting patient health."
    )
)

summary_task = Task(
    description=(
        "1. Analyze the patient's medical records in detail.\n"
        "2. Identify and prioritize the most serious conditions based on their potential impact on the patient's health and quality of life.\n"
        "3. Create a brief list of the top 3-5 most serious conditions at the beginning of the summary.\n"
        "4. For each identified condition (past, present, or potential future), starting with the most serious:\n"
        "   a. State the condition clearly.\n"
        "   b. Provide the specific medical codes (ICD-10, etc.) associated with this condition.\n"
        "   c. Explain why this condition was diagnosed or is suspected, citing specific evidence from the patient's records.\n"
        "   d. If relevant, mention any treatments or medications prescribed for this condition.\n"
        "5. For potential future conditions:\n"
        "   a. Explain the risk factors present in the patient's history that suggest this potential condition.\n"
        "   b. Provide reasoning for why this condition might develop.\n"
        "6. Organize the remaining summary into clear sections: Past Conditions, Present Conditions, and Potential Future Risks.\n"
        "7. Ensure the summary is comprehensive yet concise, aiming for around 400-500 words to accommodate the detailed analysis and prioritization."
    ),
    expected_output=(
        "A detailed summary of the patient's medical conditions, including:\n"
        "- A prioritized list of the 3-5 most serious conditions at the top\n"
        "- Clear identification of past, present, and potential future conditions, starting with the most serious\n"
        "- Specific medical codes associated with each condition\n"
        "- Evidence and reasoning for each identified condition\n"
        "- Treatments or medications, if relevant\n"
        "- Explanation of risk factors for potential future conditions\n"
        "- Organized sections for remaining conditions: Past, Present, and Potential Future Risks"
    ),
    output_file="detailed_medical_summary.md",
    agent=summary_agent,
    context=[patientRecordSelect_task]
)

commonDisease_agent = Agent(
    role="Detector of common diseases",
    goal="Find out if the given patient has diabetes and chronic kidney disease and list the relevant diagnosis codes.",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = llm,
    backstory=(
       "Diabetes and chronic kidney disease are very common, especially among older people. "
        "We are dealing with a Medicare population. Analyze a patient's medical records to see if this patient has these diseases. "
        "If the patient has any of the mentioned diseases, identify the specific ICD-10 codes that indicate these conditions. "
        "Then, suggest a treatment plan for each detected disease."
    )
)

commonDisease_task = Task(
    description=(
        "1. Analyze the patient's medical records.\n"
        "2. Answer yes or no for diabetes and chronic kidney disease."
        "3. If the answer is yes for either disease:\n"
        "   a. List the specific ICD-10 codes from the patient's records that indicate the disease.\n"
        "   b. Suggest a treatment plan and dietary restrictions. When searching the internet, make sure to check out the Mayo Clinic website.\n"
        "4. Limit the total response length to 250 words."
    ),
    expected_output="Yes or no answer to common chronic diseases, relevant ICD-10 codes if present, and suggestions if the answer is yes.",
    agent=commonDisease_agent,
    context=[patientRecordSelect_task, summary_task],
    output_file="commonDisease.md"
)

In [ ]:
chatbot_agent = Agent(
    role="Medical Information Chatbot",
    goal="Provide accurate and helpful responses to user questions about the patient's medical condition or general medical queries",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an AI-powered medical chatbot with the ability to analyze and interpret provided medical records, "
        "summaries, and assessments. You can answer questions "
        "specific to a patient as well as general medical queries. You have access to the outputs of "
        "previous analyses including patient record selection, medical summary, and common disease assessment. "
        "You can also search for additional information when needed to provide comprehensive answers."
    )
)

In [ ]:
chatbot_task = Task(
    description=(
        "As a medical chatbot, your task is to answer '{user_question}' asked by the user accurately and helpfully. Follow these steps:\n"
        "1. Review the following patient-specific information:\n"
        "   a. Patient record selection results\n"
        "   b. Medical summary\n"
        "   c. Common disease assessment\n"
        "2. For the user question:\n"
        "   a. First, attempt to answer using the patient-specific information provided.\n"
        "   b. If the patient-specific information is insufficient, then use your general medical knowledge or perform a web search.\n"
        "   c. Provide a clear, concise, and accurate answer, prioritizing patient-specific information when available.\n"
        "   d. Ensure your response is easy for a non-medical professional to understand.\n"
        "   e. If the question is outside your scope or requires immediate medical attention, advise the user to consult a healthcare professional.\n"
        "   f. If you need to make any assumptions or if there's any ambiguity, clearly state this in your response."
    ),
    expected_output="A clear, informative, and accurate response to the user's question, prioritizing patient-specific information when available.",
    agent=chatbot_agent,
    output_file="response.txt",
    context=[patientRecordSelect_task, summary_task, commonDisease_task]
)

In [ ]:
@tool
def get_diagnosis_code_details(df_list, mcid):
    """
    Get the details of unique diagnosis codes for the given patient.
    
    Args:
        df_list (list): List of dataframes containing patient medical records.
        mcid (str): Medical record ID of the patient.
    
    Returns:
        str: count of unique diagnosis code and A summary of the unique diagnosis codes and their details.
    """
    df = pd.DataFrame(df_list)
    df = df[df['mcid'] == mcid]
    
    # Get unique diagnosis codes
    unique_codes = df['cd_value'].unique()
    num_unique_codes = len(unique_codes)
    
    # Get date of service and description for each code
    code_details = {}
    for code in unique_codes:
        code_data = df[df['cd_value'] == code]
        code_details[code] = {
            'description': code_data['ICD_FULL_DESC'].iloc[0],
            'dates_of_service': code_data['dos'].tolist()
        }
    
    # Format the output
    output = f"The patient with MCID {mcid} has {num_unique_codes} unique diagnosis codes.\n\n"
    
    # List the top 10 most frequent codes and their details
    sorted_codes = sorted(code_details.items(), key=lambda x: len(x[1]['dates_of_service']), reverse=True)
    for i, (code, details) in enumerate(sorted_codes[:10], start=1):
        output += f"{i}. Code: {code}\nDescription: {details['description']}\nDates of Service: {', '.join(map(str, details['dates_of_service']))}\n\n"
    
    return output

In [ ]:
diagnosis_code_agent = Agent(
    role="Diagnosis Code Analyzer",
    goal="Analyze the diagnosis codes and their details for a given patient",
    tools=[get_diagnosis_code_details],
    verbose=True,
    llm=llm,
    backstory=(
        "As a medical professional, I have extensive experience in reviewing and interpreting patient medical records, particularly the diagnosis codes. My role is to provide a comprehensive analysis of the unique diagnosis codes found in the patient's records, including the code descriptions and the dates of service they were prescribed.")
)

In [ ]:
diagnosis_code_task = Task(
    description="Provide details about the diagnosis codes and their dates of service for the given patient",
    expected_output="A summary of the how many unique diagnosis codes present and their details",
    agent=diagnosis_code_agent,
    output_file='diag_code.md',
    context=[patientRecordSelect_task]
)

In [ ]:
patient_analysis_crew = Crew(
    agents=[patientRecordSelect_agent],
    tasks=[patientRecordSelect_task],
    verbose=2)

In [ ]:
summary_agent_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=2
)

In [ ]:
common_disease_crew =  Crew(
    agents=[summary_agent, commonDisease_agent],
    tasks=[commonDisease_task],
    verbose=2
)


In [ ]:
# Create separate Crew for chatbot
chatbot_crew = Crew(
    agents=[chatbot_agent],
    tasks=[chatbot_task],
    verbose=2
)

In [ ]:
diagnosis_code_crew = Crew(
    agents=[diagnosis_code_agent],
    tasks=[diagnosis_code_task],
    verbose=2
)

In [ ]:
# Global variables
df_list = None
mcid = None
results_store = {}

def process_mcid(mcid_input):
    global df_list, mcid, results_store
    mcid = mcid_input
    df_list = json.loads(df_json)
    
    # Run the crews
    results_store["Patient Records"] = patient_analysis_crew.kickoff({"df_list": df_list, "mcid": mcid})
    results_store["Summary"] = summary_agent_crew.kickoff({"df_list": df_list, "mcid": mcid})
    results_store["Common Diseases"] = common_disease_crew.kickoff({"df_list": df_list, "mcid": mcid})
    results_store["Diagnosis Codes"] = diagnosis_code_crew.kickoff({"df_list": df_list, "mcid": mcid})
    
    return "Analysis complete. Please check individual tabs for results."

In [ ]:
def get_patient_records():
    if "Patient Records" not in results_store:
        return "Please analyze patient records first."
    return f"Patient Records for MCID {mcid}:\n\n{results_store['Patient Records']}"

def get_summary():
    if "Summary" not in results_store:
        return "Please analyze patient records first."
    return f"Patient Summary:\n\n{results_store['Summary']}"

def get_common_diseases():
    if "Common Diseases" not in results_store:
        return "Please analyze patient records first."
    return f"Common Diseases Assessment:\n\n{results_store['Common Diseases']}"

def get_diagnosis_codes():
    if "Diagnosis Codes" not in results_store:
        return "Please analyze patient records first."
    return f"Diagnosis Codes Analysis:\n\n{results_store['Diagnosis Codes']}"

def chat_with_bot(user_question):
    if not results_store:
        return "Please enter a patient MCID and analyze records first."
    
    chatbot_response = chatbot_crew.kickoff({
        "df_list": df_list,
        "mcid": mcid,
        "user_question": user_question
    })
    return chatbot_response

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Medical Record Analysis and Chatbot")
    
    with gr.Row():
        mcid_input = gr.Textbox(label="Enter Patient MCID")
        analyze_button = gr.Button("START")
    
    analysis_output = gr.Textbox(label="Analysis Status")
    
    analyze_button.click(process_mcid, inputs=mcid_input, outputs=analysis_output)
    
    with gr.Tab("Patient Record Select"):
        records_output = gr.Markdown()
        get_records_button = gr.Button("Get Patient Records")
        get_records_button.click(get_patient_records, outputs=records_output)
        
    with gr.Tab("Diagnosis Codes"):
        diagnosis_codes_output = gr.Markdown()
        get_diagnosis_codes_button = gr.Button("Get Diagnosis Codes")
        get_diagnosis_codes_button.click(get_diagnosis_codes, outputs=diagnosis_codes_output)
    
    with gr.Tab("Summary"):
        summary_output = gr.Markdown()
        get_summary_button = gr.Button("Get Patient Summary")
        get_summary_button.click(get_summary, outputs=summary_output)
    
    with gr.Tab("Common Disease"):
        diseases_output = gr.Markdown()
        get_diseases_button = gr.Button("Get Common Diseases Assessment")
        get_diseases_button.click(get_common_diseases, outputs=diseases_output)
    
    with gr.Tab("Chat with Medical Bot"):
        chat_input = gr.Textbox(label="Ask a question about the patient or any medical query")
        chat_button = gr.Button("Ask")
        chat_output = gr.Markdown()
        
        chat_button.click(chat_with_bot, inputs=chat_input, outputs=chat_output)

demo.launch(share=True)